In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Mitochondrial_Disorders/GSE65399'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Epigenetic therapy for Friedreich ataxia."
!Series_summary	"We set out to investigate whether a histone deacetylase inhibitor (HDACi) would be effective in an in vitro model for the neurodegenerative disease Friedreich ataxia (FRDA) and to evaluate safety and surrogate markers of efficacy in a phase I clinical trial in patients. In the neuronal cell model, HDACi 109/RG2833 increases FXN mRNA levels and frataxin protein, with concomitant changes in the epigenetic state of the gene. Chromatin signatures indicate that histone H3 lysine 9 is a key residue for gene silencing through methylation and reactivation through acetylation, mediated by the HDACi. Drug treatment in FRDA patients demonstrated increased FXN mRNA and H3 lysine 9 acetylation in peripheral blood mononuclear cells. No safety issues were encountered."
!Series_overall_design	"We used a human FRDA neuronal cell model, derived from patient induced pluripotent stem cells, to determine the 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Check gene expression data availability
is_gene_available = True

# Step 2: Variable Availability and Data Type Conversion

# Evaluate 'Mitochondrial_Disorders' availability
# This dataset description suggests there is no explicit mention of mitochondrial disorders
trait_row = None

# Evaluate 'age' availability
if 1 in sample_characteristics_dict:
    age_values = sample_characteristics_dict[1]
    unique_age_values = set(age_values)
    if len(unique_age_values) > 1:
        age_row = 1

# Evaluate 'gender' availability
# Skipping since no mentioned gender-related data in the given output
gender_row = None

# Step 2.3: Define conversion functions
def convert_trait(value):
    return None

def convert_age(value):
    try:
        val = value.split(':')[-1].strip()
        if val.lower() == 'nan':
            return None
        return float(val.rstrip('wk').strip('d')) if 'wk' in val or 'd' in val else float(val)
    except:
        return None

def convert_gender(value):
    return None

# Step 3: Save Metadata
save_cohort_info('GSE65399', './preprocessed/Mitochondrial_Disorders/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
# Skipping because trait_row is None (No clinical data available)
